Weighted Mean as "Statistic"

- Computes a mean using a set of weights. These weights need not sum to one.
- We are seeking how the weighted mean changes with the weights, keeping the datapoints fixed.


In [44]:
import numpy as np
np.random.seed(43)
n = 100
x = np.random.uniform(0,1,n)

# Statistics
sample_mean = np.mean(x)
sample_sd = np.std(x)
sample_se = sample_sd/np.sqrt(n)

# Influence function for each datapoint
L = x - sample_mean
print(L[0:5])
print(np.mean(L), np.sqrt(np.var(L)/n))

def weightedMean(x,w):
    return np.sum(x*w)/np.sum(w)

print(weightedMean(x,np.arange(100)))
print(weightedMean(x,np.ones(100)/n))

[-0.40247867  0.0915333  -0.38414227 -0.27694362 -0.19039418]
1.4432899320127036e-17 0.02913158201793312
0.5292329793628652
0.5175332356349095


### Jacknife Influence Function

- We seek a vector of values that tell us the effect of a particular datapoint on the "statistic".
- Jacknife is a resampling technique used to nonparametrically obtain sampling variance and bias in estimates of a population parameter
- Given sample $\{x_i\}^n{i=1}$ and an estimator $\hat{\theta}$ of parameter $\theta$, we build $n$ samples by leaving one observation out and compute $\{\hat{\theta}_i\}^n_{i=0}$ (pseudoestimate) on these subsamples.
- The jacknife estimator is,

$$\bar{\theta} = (1/n) \sum_i \hat{\theta}_i$$

- The  "jacknife influence function" (pseudovalues) or the influence of one point on the statistic is, where the $n-1$ scales the effect to the size of the sample remaining.

$$b_i = (n-1)(\bar{\theta} - \hat{\theta}_i)$$

- The bias in $\hat{\theta}$ is estimated by, because this turns out to be an unbiased estimator for most statistics. The way to prove this is to show that if the statistic is biased, then by takign this estimate of bias we can approximate the bias up to second order terms.

$$\bar{b} = (n-1)(\bar{\theta}- \hat{\theta})$$

- using this a correction is made, giving us the "bias-corrected jacknife estimate",

$$\tilde{\theta} = \hat{\theta} - \bar{b} = n \hat{\theta} - (n-1) \bar{\theta}$$

- we also get the estimate for the sampling variance of $\hat{\theta}$ via,

$$\hat{\sigma}_{\hat{\theta}} = \frac{n}{n-1} \sum_i (\hat{\theta}_i - \hat{\theta})^2 $$

Note that if the estimator is unbiased, then the bias-corrected estimates will be the same as the original estimator.

In [42]:
def jacknife(data, statistic):
    """Computes the jacknife influence function from data and statistic"""
    n = data.shape[0]
    w_orig = np.ones(n)*(1/n)
    T_obs = statistic(data, w_orig)
    L = np.empty(n)
    for i in range(n):
        w = w_orig.copy()
        w[i] = 0
        L[i] = (n-1) * (T_obs - statistic(data,w)) # Estimates of the bias i.e. the "jacknife influence function"
    return L

Lj = jacknife(x, weightedMean)
print('Avg bias:', np.mean(Lj), 'S.e. of Bias:', np.sqrt(np.var(Lj)/n))
print('Jacknife Influence Function \n', Lj)

Avg bias: 1.0331735467161707e-14 S.e. of Bias: 0.029131582017932676
Jacknife Influence Function 
 [-0.40247867  0.0915333  -0.38414227 -0.27694362 -0.19039418  0.34160426
  0.14855698  0.02362898 -0.48851941  0.21621506 -0.12258322  0.28451388
 -0.26311211 -0.4606483   0.34911541 -0.29650424 -0.11254379 -0.20143676
 -0.44087054  0.32569146  0.33140591  0.45393186 -0.13215632  0.43695489
 -0.07177487  0.15219142 -0.43503319  0.37956534 -0.21952973 -0.25522841
 -0.51240369  0.02566928 -0.04193687  0.11884044  0.4606709   0.39112952
  0.39261984  0.00772244 -0.41351429 -0.33661864  0.43550699 -0.10558025
  0.34748388  0.15464404  0.11125256 -0.24197446  0.37921403 -0.31064186
 -0.11312799  0.47603926  0.21819385 -0.07247182  0.04312993 -0.10627775
  0.20945476 -0.11833635  0.15261193  0.18718237  0.09202664  0.02250123
 -0.31145138 -0.31837176  0.27820563 -0.22720046  0.13842959 -0.21791646
 -0.37305486 -0.11357657 -0.20726371 -0.27413522  0.07057081 -0.27218998
  0.23023738  0.20261341  

In [43]:
def infinitsmalJacknife(data, statistic, eps=0.001):
    n = data.shape[0]
    eps = eps/n
    w_orig = np.ones(n)/n
    T_obs = statistic(data, w_orig)
    L = np.empty(n)
    for i in range(n):
        w = w_orig.copy()
        w[i] = w[i] + eps
        L[i] = n * (statistic(data,w)-T_obs) / eps
    return L

print('Jacknife Infinitsmal Influence Function \n', infinitsmalJacknife(x, weightedMean))

Jacknife Infinitsmal Influence Function 
 [-40.24746445   9.15323883 -38.41384301 -27.69408463 -19.03922759
  34.16008393  14.85554919   2.36287404 -48.85145261  21.62128985
 -12.25819914  28.45110379 -26.31094787 -46.06436925  34.91119141
 -29.65012747 -11.25426622 -20.14347503 -44.0866127   32.56882017
  33.14025981  45.39273165 -13.21549997  43.695052    -7.17741571
  15.21898942 -43.50288357  37.95615487 -21.95275397 -25.52258608
 -51.23985626   2.5669023   -4.19364493  11.88392535  46.06662918
  39.11256109  39.2615914    0.77223582 -41.35101507 -33.66152742
  43.55026303 -10.55791965  34.74804102  15.46424974  11.12514477
 -24.19720393  37.9210242  -31.06387579 -11.31268613  47.60344964
  21.81916639  -7.24710984   4.31295025 -10.62766838  20.94526618
 -11.83351664  15.26104029  18.71805004   9.20257154   2.25010035
 -31.14482703 -31.83685753  27.82028471 -22.71981859  13.84282079
 -21.79142807 -37.30511267 -11.35754329 -20.72616362 -27.4132478
   7.05701013 -27.21872608  23.0235

### Automatic Differentiation

$$y = z^2 + z$$
$$\frac{dy}{dz}(z) = 2z + 1$$

If $z = 0.5$, then $\frac{dy}{dz}(0.5) = 2$.

In [10]:
import torch
z = torch.tensor(0.5,requires_grad = True)
y = z**2 + z
y.backward()
z.grad

tensor(2.)

### Jacknife Influence Functions and Autograd

The statistic is a function of data $x$ and weights $w$. The influence function of $x=\bar{x}$, is the change in statistic when the weight of the data point $x=\bar{x}$ is increased by 1 unit.
$$\hat{\theta} = f(x,w)$$
$$\frac{d\hat{\theta}}{dw}(\bar{x}) = f_w(\bar{x},w)$$
This is exactly what infinitsmal jacknife tries to get at.

In [38]:
n = 100
x = torch.randn((1,n))
w = torch.ones(n)/n
w.requires_grad = True
mu = torch.matmul(x,w)/torch.sum(w)
grad = torch.autograd.grad(mu,w)
print(grad)

(tensor([ 0.7324, -0.5506, -0.2545,  0.1038, -0.8007, -1.0531, -0.4438, -0.8725,
        -1.2087,  1.6367,  0.3946, -0.5195, -0.7956, -0.0837,  0.7275,  0.4463,
        -0.9485, -0.5870,  1.6192, -0.0712, -0.6058, -1.2057, -0.6172,  0.9792,
         2.5363, -0.2663, -0.6411, -1.4042, -0.0191,  1.9239,  0.2427,  0.5955,
         0.5785, -2.1879,  0.7579,  0.5688,  0.5513,  1.0324,  0.9223, -0.5409,
        -1.6105,  0.2276, -1.8005, -0.9163, -1.5590,  0.2659,  1.7567, -0.3657,
        -0.3887, -0.8885,  0.3104, -0.8025,  0.2945, -1.4920,  1.6627, -0.2443,
        -0.5787,  0.6156, -0.0587,  1.1298, -0.0226, -3.0926, -0.7761,  0.0301,
         3.2803,  0.2875, -0.5157,  0.8228,  1.3792, -2.4762, -1.9655,  1.7365,
         0.1031, -0.1755, -0.2997, -1.1121,  0.2728,  1.0846,  0.3280, -0.1043,
        -0.3186,  0.9809, -0.5722,  0.1787,  1.3046,  0.3455,  1.0404, -0.8022,
        -0.4855,  1.4789, -0.6434,  0.7033,  0.0740,  1.3402,  0.4179,  1.1582,
        -1.4108,  0.7467, -1.5314,  0.9

In [39]:
infinitsmalJacknife(x.detach().numpy().reshape(-1), weightedMean)

array([  73.23481426,  -55.05508059,  -25.45060788,   10.38415459,
        -80.0739933 , -105.31168266,  -44.38443001,  -87.24723924,
       -120.87216596,  163.67068439,   39.46217976,  -51.94657448,
        -79.55676128,   -8.36603192,   72.74940484,   44.63025609,
        -94.84891312,  -58.70138212,  161.91483256,   -7.11711625,
        -60.58236073, -120.57118936,  -61.71867615,   97.91864492,
        253.62408955,  -26.63196907,  -64.10497789, -140.42091113,
         -1.91413113,  192.3832898 ,   24.26861004,   59.55315786,
         57.84646745, -218.78768261,   75.78621325,   56.8805302 ,
         55.13043016,  103.23685193,   92.23389089,  -54.08837445,
       -161.04794123,   22.76441396, -180.05216765,  -91.63330449,
       -155.89933158,   26.58475781,  175.67125628,  -36.57323895,
        -38.87037899,  -88.8457781 ,   31.04068497,  -80.24981332,
         29.44976052, -149.20220874,  166.2650579 ,  -24.42514407,
        -57.86771817,   61.55469758,   -5.86721265,  112.98297